<a href="https://colab.research.google.com/github/Suren1206/House_Price_Prediction/blob/main/001_HPP_Clean_07132025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## to have the csv data file worked in collab first it has to be imported into collab.
from google.colab import files
uploaded = files.upload()

import pandas as pd
train_df = pd.read_csv('train.csv')

##Add few useful features based on some redundant fields available in dataset

# new feature using 'yr sold' to determine the age of the house and garage
train_df['HouseAge'] = train_df['YrSold'] - train_df['YearBuilt']
train_df['GarageYrBlt'] = train_df['GarageYrBlt'].fillna(train_df['YearBuilt'])
train_df['GarageAge'] = train_df['YrSold'] - train_df['GarageYrBlt']
train_df['RemodAge'] = train_df['YrSold'] - train_df['YearRemodAdd']

# On verification of the 3 created fields as above, there was a need to finetune a particular record for RemodAge

train_df.loc[train_df['Id'] == 524, 'YearRemodAdd'] = train_df.loc[train_df['Id'] == 524, 'YrSold'].values[0]
train_df['RemodAge'] = train_df['YrSold'] - train_df['YearRemodAdd']

# Add a flag for 2008Q1 sales which is proved to be 11% over prediction based on trend analysis
train_df['Flag_2008Q1'] = (
    (train_df['YrSold'] == 2008) &
    (train_df['MoSold'].isin([1, 2, 3]))
).astype(int)

# Add a flag for AbnormalSale where transaction types are unusual
train_df['Flag_AbnormalSale'] = train_df['SaleCondition'].isin(['Normal', 'Partial'])
train_df['Flag_AbnormalSale'] = train_df['Flag_AbnormalSale'].astype(int)

## Look at all numeric fields for NaaN values & fill it up with imputed values/median/0 depending on the case
#(A) Handle Null value in LotFrontage # compare with related fields and take a call
# Step 1: Create a median map for each (LotShape, LotConfig) group
median_map = (
    train_df.groupby(['LotShape', 'LotConfig'])['LotFrontage']
    .median()
    .reset_index()
    .rename(columns={'LotFrontage': 'MedianFrontage'})
)
# Step 2: Merge the median values back into the main DataFrame
train_df = train_df.merge(median_map, on=['LotShape', 'LotConfig'], how='left')
# Step 3: Fill missing LotFrontage values using the group median
train_df['LotFrontage'] = train_df['LotFrontage'].fillna(train_df['MedianFrontage'])
# Step 4: Drop the helper column
train_df.drop(columns=['MedianFrontage'], inplace=True)
# Step 5 : verify again and fill up the lone record where it could not pick up median value based on reference by giving overall median
train_df['LotFrontage'] = train_df['LotFrontage'].fillna(train_df['LotFrontage'].median())
#(B) Handle Null value in MasVnrType  # compare with related fields & take a decision
train_df.loc[train_df['MasVnrType'].isnull(), 'MasVnrType'] = 'None'
train_df.loc[train_df['MasVnrArea'].isnull(), 'MasVnrArea'] = 0.0

## Look at all non numeric fields for NaaN values & fill it up with "None" after verifying its related fields
garage_cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond']
train_df[garage_cols] = train_df[garage_cols].fillna('None')
basement_cols = ['BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2']
train_df[basement_cols] = train_df[basement_cols].fillna('None')
others_cols = ['PoolQC', 'Fence', 'MiscFeature', 'Alley', 'FireplaceQu', 'Electrical']
train_df[others_cols] = train_df[others_cols].fillna('None')

## Look for wierd entries - N/A, na or #n/a etc., from entire list & replace them with the data description file
train_df['MSZoning'] = train_df['MSZoning'].replace({'C (all)': 'C'})

## check for skewness and outliers - list of columns identified for log conversion seperately for each & merged

import numpy as np
# Fields to log-transform
log_fields = [
    'BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MSSubClass',
    'LotArea', 'MasVnrArea', 'OpenPorchSF', 'BsmtFinSF1', 'WoodDeckSF',
    'TotalBsmtSF', 'LotFrontage', 'SalePrice', '1stFlrSF',
    'GrLivArea', 'KitchenAbvGr'
]
# Apply log1p transformation and add new columns to train_df
for col in log_fields:
    if col in train_df.columns:
        train_df[f"{col}_log"] = np.log1p(train_df[col])
    else:
        print(f"⚠️ Column not found in DataFrame: {col}")


## Apply one hot encoded to various category based values

categorical_fields = [
    'MSSubClass','MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
    'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1',
    'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl',
    'Exterior1st', 'Exterior2nd', 'MasVnrType', 'Foundation', 'Heating',
    'CentralAir', 'Electrical', 'Functional', 'GarageType', 'GarageFinish',
    'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType'
]

# One-hot encode with drop_first=False (keep all categories for full analysis)
train_df_encoded = pd.get_dummies(train_df, columns=categorical_fields, drop_first=False)

## for all perception based fields, create SCORE where ever needed & do a correlation

# Common score mapping for quality/condition fields
quality_score_map = {
    'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': -1, 'NA': 0, np.nan: 0
}

# BsmtExposure mapping
bsmt_exposure_map = {
    'Gd': 3, 'Av': 2, 'Mn': 1, 'No': 0, 'NA': 0, np.nan: 0
}

# BsmtFinType mapping
bsmt_fin_type_map = {
    'GLQ': 3, 'ALQ': 2, 'BLQ': 1, 'Rec': 1, 'LwQ': -1, 'Unf': 0, 'NA': 0, np.nan: 0
}
# Fields using the common quality score
quality_fields = [
    'ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
    'HeatingQC', 'KitchenQual', 'FireplaceQu',
    'GarageQual', 'GarageCond'
]


# Apply quality score mapping
for field in quality_fields:
    train_df_encoded[field + '_Score'] = train_df_encoded[field].map(quality_score_map)

# Apply BsmtExposure score
train_df_encoded['BsmtExposure_Score'] = train_df_encoded['BsmtExposure'].map(bsmt_exposure_map)

# Apply BsmtFinType1 and 2 score
train_df_encoded['BsmtFinType1_Score'] = train_df_encoded['BsmtFinType1'].map(bsmt_fin_type_map)
train_df_encoded['BsmtFinType2_Score'] = train_df_encoded['BsmtFinType2'].map(bsmt_fin_type_map)

# List all _Score columns
score_cols = [col for col in train_df_encoded.columns if col.endswith('_Score')]

# Replace NaNs in score fields with 0
train_df_encoded[score_cols] = train_df_encoded[score_cols].fillna(0)


## Do a correlation seperately for each category - numerical fields, one hot encoded fields & perception / rank based to get the list of features

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Final feature list (copy-pasted from your list)
final_features = [
    'ExterQual_Score', 'BsmtQual_Score', 'KitchenQual_Score', 'FireplaceQu_Score',
    'OverallQual', 'MSSubClass_60', 'Foundation_PConc', 'GarageType_Attchd',
    'GarageFinish_Fin', 'GarageFinish_Unf', '1stFlrSF_log', 'GrLivArea_log',
    'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'HouseAge', 'RemodAge','Flag_AbnormalSale'
]

# Define features (X) and target (y)
X = train_df_encoded[final_features]
y = train_df_encoded['SalePrice_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

# Evaluation metrics
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"✅ RMSE: {rmse:.4f}")
print(f"🔍 R-squared: {r2:.4f}")

Saving train.csv to train.csv
✅ RMSE: 0.1580
🔍 R-squared: 0.8663
